In [168]:
from parsl import *

import logging
import sys
import concurrent.futures as cf
import kale.workflow_objects as wo
from kale.workflow_widgets import WorkflowWidget, WorkerPoolWidget

In [180]:
wpw = WorkerPoolWidget()
wpw

WorkerPoolWidget(children=(HTML(value='<h3>Worker Pools</h3>'), TableWidget(children=(RowWidget(children=(HBox(children=(HTML(value='<b>Name</b>'),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(HTML(value='<b>Location</b>'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='<b>Executor</b><b>Workers</b>'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='<b>Action</b>'),), layout=Layout(justify_content='flex-start', width='60px'))), layout=Layout(justify_content='space-between', width='710px')), RowWidget(children=(HBox(children=(HTML(value='default'),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(HTML(value='localhost'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='parsl'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='4'),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(Button(button_style='danger', description='Remove', style=ButtonStyle()),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px')), RowWidget(children=(HBox(children=(Text(value='', layout=Layout(width='148px')),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(Dropdown(layout=Layout(width='198px'), options=('localhost',), value='localhost'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(Dropdown(layout=Layout(width='198px'), options=('parsl', 'fireworks'), value='parsl'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(IntText(value=1, layout=Layout(width='198px')),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(Button(button_style='success', icon='plus', style=ButtonStyle()),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px'))), layout=Layout(width='710px')), HTML(value='\n        <div class="alert alert-success" style="width: 710px">\n        WorkerPool \'default\' created.\n        </div>\n        ')))

In [176]:
class KaleDFK(DataFlowKernel):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fu_to_func = dict()
        self.fu_to_task = dict()
        
        # A workflow must be created manually
        # by calling self.new_workflow(name)
        self.workflow_created = False
        
        # Store names and funcs of deps,
        # indexed by task_id.
        self.dep_funcs = dict()
        self.dep_names = dict()
        
    def parse_args(self, args, kwargs):
        """Replace any futures in args or kwargs with their corresponding Task at definition time
        (which will be replaced by the future's result by Kale at execution time)."""
        new_args = []
        for arg in args:
            if isinstance(arg, cf.Future):
                new_args.append(self.fu_to_task[arg])
            else:
                new_args.append(arg)

        new_kwargs = dict()
        for key, val in kwargs.items():
            if isinstance(val, cf.Future):
                new_kwargs[key] = self.fu_to_task[val]
            else:
                new_kwargs[key] = val

        return new_args, new_kwargs

    
    def new_workflow(self, name):
        self.kale_workflow = wo.Workflow(name=name)
        self.workflow_created = True
    
    def submit(self, func, *args, **kwargs):
        if self.workflow_created:
            return self.inner_submit(func, *args, **kwargs)
        else:
            raise Exception("Call KaleDFK.new_workflow(name) before adding tasks.")
        
    def inner_submit (self, func, *args, **kwargs):
        from parsl.dataflow.states import States
        from parsl.dataflow.futures import AppFuture
        ''' Add task to the dataflow system.

        If all deps are met :
              send to the runnable queue
              and launch the task
        Else:
              post the task in the pending queue

        Returns:
               (AppFuture) [DataFutures,]
        '''
        
        print("""Inner submit
        self: {}
        func: {}
        args: {}
        kwargs: {}
        """.format(self,func, args, kwargs))
        

        task_id = self.task_count
        self.task_count += 1
        
        # Extract task & dependency info
        task_name = func.__name__

        dep_cnt, depends = super()._count_all_deps(task_id, args, kwargs)

        print("task_id = {}".format(task_id))
        print("args = {}".format(args))
        print("kwargs = {}".format(kwargs))
        print("dep_cnt = {}".format(dep_cnt))
        print("depends = {}".format(depends))

        
        self.dep_funcs[task_id] = [self.fu_to_func[fu] for fu in depends]
        self.dep_names[task_id] = [dep.__name__ for dep in self.dep_funcs[task_id]]

        print("dep funcs = {}".format(self.dep_funcs[task_id]))
        print("dep names = {}".format(self.dep_names[task_id]))
        
        #dep_cnt  = self._count_deps(dep ends, task_id)
        task_def = { 'depends'    : depends,
                     'func'       : func,
                     'args'       : args,
                     'kwargs'     : kwargs,
                     'callback'   : None,
                     'dep_cnt'    : dep_cnt,
                     'exec_fu'    : None,
                     'status'     : States.unsched,
                     'app_fu'     : None  }

        if task_id in self.tasks:
            raise DuplicateTaskError("Task {0} in pending list".format(task_id))
        else:
            self.tasks[task_id] = task_def

        # Extract s
        # Create Task
        #task = wo.PythonFunctionTask()tdout and stderr to pass to AppFuture:
        task_stdout = kwargs.get('stdout', None)
        task_stderr = kwargs.get('stderr', None)

        if dep_cnt == 0 :
            # Set to running
            new_args, kwargs, exceptions = self.sanitize_and_wrap(task_id, args, kwargs)
            if not exceptions:
                self.tasks[task_id]['exec_fu'] = self.launch_task(task_id, func, *new_args, **kwargs)
                self.tasks[task_id]['app_fu']  = AppFuture(self.tasks[task_id]['exec_fu'],
                                                           tid=task_id,
                                                           stdout=task_stdout,
                                                           stderr=task_stderr)
                self.tasks[task_id]['status']  = States.running
            else:
                self.tasks[task_id]['exec_fu'] = None
                app_fu = AppFuture(self.tasks[task_id]['exec_fu'],
                                   tid=task_id,
                                   stdout=task_stdout,
                                   stderr=task_stderr)
                app_fu.set_exception(DependencyError(exceptions, "Failures in input dependencies", None))
                self.tasks[task_id]['app_fu']  = app_fu
                self.tasks[task_id]['status']  = States.dep_fail
        else:
            # Send to pending, create the AppFuture with no parent and have it set
            # when an executor future is available.
            self.tasks[task_id]['app_fu']  = AppFuture(None, tid=task_id,
                                                       stdout=task_stdout,
                                                       stderr=task_stderr)
            self.tasks[task_id]['status']  = States.pending

        #logger.debug("Task:%s Launched with AppFut:%s", task_id, task_def['app_fu'])
        
        
        fu = task_def['app_fu'] # This was the return value
        
        ## End of Parsl code
        
        self.fu_to_func[fu] = func
        
        # Replace futures with Tasks in args for Task definition
        new_args, new_kwargs = self.parse_args(args, kwargs)
        
        # Create Task
        task = wo.PythonFunctionTask(
            name=task_name,
            func=func,
            args=new_args,
            kwargs=new_kwargs
        )
        
        self.fu_to_task[fu] = task
        
        dep_tasks = [
            self.kale_workflow.get_task_by_name(dep_name)
            for dep_name in self.dep_names[task_id]
        ]
        
        # Add to workflow
        self.kale_workflow.add_task(
            task,
            dependencies=dep_tasks
        )
        
        return fu
        


In [189]:
workers = cf.ThreadPoolExecutor()
parsl_dfk = DataFlowKernel(executors=[workers])
kale_dfk = KaleDFK(executors=[workers])
kale_dfk.new_workflow('MyWorkflow')

In [190]:
@App('python', kale_dfk)
def rand_add(prev, lim=[0,10]):
    """Add a random number to the previous one.
    """
    import random
    myrand = random.randint(*lim)
    mysum = myrand + prev
    print("My number is {}. Theirs was {}. The sum is {}.".format(myrand, prev, mysum))
    return mysum

In [191]:
rand_add(rand_add(0))

Inner submit
        self: <__main__.KaleDFK object at 0x7f815c10ec50>
        func: <function rand_add at 0x7f815c0f9598>
        args: (0,)
        kwargs: {}
        
task_id = 0
args = (0,)
kwargs = {}
dep_cnt = 0
depends = []
dep funcs = []
dep names = []
My number is 4. Theirs was 0. The sum is 4.Inner submit
        self: <__main__.KaleDFK object at 0x7f815c10ec50>
        func: <function rand_add at 0x7f815c0f9598>
        args: (<AppFuture at 0x7f815c09fac8 state=running>,)
        kwargs: {}
        

task_id = 1
args = (<AppFuture at 0x7f815c09fac8 state=finished returned int>,)
kwargs = {}My number is 5. Theirs was 4. The sum is 9.

dep_cnt = 0
depends = [<AppFuture at 0x7f815c09fac8 state=finished returned int>]
dep funcs = [<function rand_add at 0x7f815c0f9598>]
dep names = ['rand_add']


<AppFuture at 0x7f815c09feb8 state=finished returned int>

In [192]:
WorkflowWidget(kale_dfk.kale_workflow, wpw)

WorkflowWidget(children=(VBox(children=(Figure(fig_margin={'left': 10, 'right': 10, 'bottom': 10, 'top': 10}, layout=Layout(height='600px', width='400px'), marks=[Graph(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], highlight_links=False, interactions={'hover': 'tooltip', 'click': 'select'}, link_data=[{'source': 0, 'target': 1}], link_type='line', node_data=[{'label': '0', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': [], 'input_files': [], 'output_files': [], 'num_cores': 1}, {'label': '1', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': [], 'input_files': [], 'output_files': [], 'num_cores': 1}], scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'link_color': {'dimension': 'link_color'}}, selected_style={'stroke': 'red'}, tooltip_style={'opacity': 0.9}, x=array([ 335.37,  335.37]), y=array([ 90.,  18.]))], min_aspect_ratio=0.0, scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), Toolbar(figure=Figure(fig_margin={'left': 10, 'right': 10, 'bottom': 10, 'top': 10}, layout=Layout(height='600px', width='400px'), marks=[Graph(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], highlight_links=False, interactions={'hover': 'tooltip', 'click': 'select'}, link_data=[{'source': 0, 'target': 1}], link_type='line', node_data=[{'label': '0', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': [], 'input_files': [], 'output_files': [], 'num_cores': 1}, {'label': '1', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': [], 'input_files': [], 'output_files': [], 'num_cores': 1}], scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'link_color': {'dimension': 'link_color'}}, selected_style={'stroke': 'red'}, tooltip_style={'opacity': 0.9}, x=array([ 335.37,  335.37]), y=array([ 90.,  18.]))], min_aspect_ratio=0.0, scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))))), Tab(children=(VBox(children=(HTML(value='<b>Workflow Description</b>'), EditHTML(children=(HTMLMath(value='', layout=Layout(border='1px lightgray solid', padding='10px')), Button(description='Edit Description', style=ButtonStyle()))), Box(layout=Layout(height='20px', width='0px')), HTML(value='<b>Worker Pools</b>'), VBox(children=(Dropdown(options=(('default', <kale.workflow_objects.WorkerPool object at 0x7f815c3f99e8>),), value=<kale.workflow_objects.WorkerPool object at 0x7f815c3f99e8>), Button(button_style='success', description='Run Workflow', style=ButtonStyle()))))), VBox(children=(HTML(value='<b>Task Description</b>'), EditHTML(children=(HTMLMath(value='None selected.', layout=Layout(border='1px lightgray solid', padding='10px')), Button(description='Edit Description', disabled=True, style=ButtonStyle()))), Box(layout=Layout(height='20px', width='0px')), HTML(value=''), Button(button_style='info', description='Continue Workflow', layout=Layout(visibility='hidden'), style=ButtonStyle()), Box(layout=Layout(height='20px', width='0px')), HTML(value='<b>Task Metadata</b>'), HTML(value=''))), VBox(children=(HTML(value='<b>Tags</b>'), Select(options=(), value=None), HTML(value='<b>Actions</b>'), Text(value='', description='Tag name'), HBox(children=(VBox(children=(Button(description='Select All', style=ButtonStyle()), Button(description='Select None', style=ButtonStyle()), Button(description='Select Children', style=ButtonStyle()), Button(description='Select Parents', style=ButtonSty

Workflow submitted.Attempting to start job.

parsl_run

My number is 6. Theirs was 0. The sum is 6.

My number is 3. Theirs was 6. The sum is 9.
Workflow submitted.Attempting to start job.

parsl_run

My number is 2. Theirs was 0. The sum is 2.

My number is 8. Theirs was 2. The sum is 10.


In [178]:
@App('python', kale_dfk)
def step1():
    import timedef rand_add(prev, lim=[0,10]):
    """Add a random number to the previous one.
    """
    import random
    myrand = random.randint(*lim)
    mysum = myrand + prev
    print("My number is {}. Theirs was {}. The sum is {}.".format(myrand, prev, mysum))
    return mysum
    print("Step 1")
    time.sleep(1)
    return 1
    
@App('python', kale_dfk)
def step2(prev, fav_num):
    print("Step 2 after s@App('python', kale_dfk)
def rand_add(prev, lim=[0,10]):
    """Add a random number to the previous one.
    """
    import random
    myrand = random.randint(*lim)
    mysum = myrand + prev
    print("My number is {}. Theirs was {}. The sum is {}.".format(myrand, prev, mysum))
    return mysumtep {}. My favorite number is {}.".format(prev, fav_num))
    return 2

# Outstanding Questions
- How to prevent execution at definition time?
- When to create workflow, how to pass it around to call add_task?

In [179]:
step2(step1(), 2).result()

Inner submit
        self: <__main__.KaleDFK object at 0x7f815c4b7550>
        func: <function step1 at 0x7f815e8c2c80>
        args: ()
        kwargs: {}
        
task_id = 0
args = ()
kwargs = {}
dep_cnt = 0
depends = []
dep funcs = []
dep names = []
Inner submit
        self: <__main__.KaleDFK object at 0x7f815c4b7550>
        func: <function step2 at 0x7f815e8c2a60>
        args: (<AppFuture at 0x7f815c423ac8 state=running>, 2)
        kwargs: {}
        
Step 1task_id = 1

args = (<AppFuture at 0x7f815c423ac8 state=running>, 2)
kwargs = {}
dep_cnt = 1
depends = [<AppFuture at 0x7f815c423ac8 state=running>]
dep funcs = [<function step1 at 0x7f815e8c2c80>]
dep names = ['step1']
Step 2 after step 1. My favorite number is 2.


2

In [181]:
WorkflowWidget(kale_dfk.kale_workflow, wpw)

WorkflowWidget(children=(VBox(children=(Figure(fig_margin={'left': 10, 'right': 10, 'bottom': 10, 'top': 10}, layout=Layout(height='600px', width='400px'), marks=[Graph(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], highlight_links=False, interactions={'hover': 'tooltip', 'click': 'select'}, link_data=[{'source': 0, 'target': 1}], link_type='line', node_data=[{'label': '0', 'shape': 'rect', 'name': 'step1', 'task_type': 'PythonFunctionTask', 'tags': [], 'input_files': [], 'output_files': [], 'num_cores': 1}, {'label': '1', 'shape': 'rect', 'name': 'step2', 'task_type': 'PythonFunctionTask', 'tags': [], 'input_files': [], 'output_files': [], 'num_cores': 1}], scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'link_color': {'dimension': 'link_color'}}, selected_style={'stroke': 'red'}, tooltip_style={'opacity': 0.9}, x=array([ 337.97,  337.97]), y=array([ 90.,  18.]))], min_aspect_ratio=0.0, scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), Toolbar(figure=Figure(fig_margin={'left': 10, 'right': 10, 'bottom': 10, 'top': 10}, layout=Layout(height='600px', width='400px'), marks=[Graph(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], highlight_links=False, interactions={'hover': 'tooltip', 'click': 'select'}, link_data=[{'source': 0, 'target': 1}], link_type='line', node_data=[{'label': '0', 'shape': 'rect', 'name': 'step1', 'task_type': 'PythonFunctionTask', 'tags': [], 'input_files': [], 'output_files': [], 'num_cores': 1}, {'label': '1', 'shape': 'rect', 'name': 'step2', 'task_type': 'PythonFunctionTask', 'tags': [], 'input_files': [], 'output_files': [], 'num_cores': 1}], scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'link_color': {'dimension': 'link_color'}}, selected_style={'stroke': 'red'}, tooltip_style={'opacity': 0.9}, x=array([ 337.97,  337.97]), y=array([ 90.,  18.]))], min_aspect_ratio=0.0, scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))))), Tab(children=(VBox(children=(HTML(value='<b>Workflow Description</b>'), EditHTML(children=(HTMLMath(value='', layout=Layout(border='1px lightgray solid', padding='10px')), Button(description='Edit Description', style=ButtonStyle()))), Box(layout=Layout(height='20px', width='0px')), HTML(value='<b>Worker Pools</b>'), VBox(children=(Dropdown(options=(('default', <kale.workflow_objects.WorkerPool object at 0x7f815c3f99e8>),), value=<kale.workflow_objects.WorkerPool object at 0x7f815c3f99e8>), Button(button_style='success', description='Run Workflow', style=ButtonStyle()))))), VBox(children=(HTML(value='<b>Task Description</b>'), EditHTML(children=(HTMLMath(value='None selected.', layout=Layout(border='1px lightgray solid', padding='10px')), Button(description='Edit Description', disabled=True, style=ButtonStyle()))), Box(layout=Layout(height='20px', width='0px')), HTML(value=''), Button(button_style='info', description='Continue Workflow', layout=Layout(visibility='hidden'), style=ButtonStyle()), Box(layout=Layout(height='20px', width='0px')), HTML(value='<b>Task Metadata</b>'), HTML(value=''))), VBox(children=(HTML(value='<b>Tags</b>'), Select(options=(), value=None), HTML(value='<b>Actions</b>'), Text(value='', description='Tag name'), HBox(children=(VBox(children=(Button(description='Select All', style=ButtonStyle()), Button(description='Select None', style=ButtonStyle()), Button(description='Select Children', style=ButtonStyle()), Button(description='Select Parents', style=ButtonStyle()), Butto

Attempting to start job.Workflow submitted.
parsl_run


Step 1

Step 2 after step 1. My favorite number is 2.
Workflow submitted.Attempting to start job.

parsl_run

Step 1

Step 2 after step 1. My favorite number is 2.


In [3]:
def gen_empty_future():
    f = cf.Future()
    f._state = 'FINISHED'
    return f

In [4]:
class LogFuture(cf.Future):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._state = 'FINISHED'
    
    def add_done_callback(self, *args, **kwargs):
        print("Done callback:")
        print("args = {}".format(args))
        print("kwargs = {}".format(kwargs))
        super().add_done_callback(*args, **kwargs)